# crawling berita di kompas

## apa itu crawling?

crawling merupakan program otomastis untuk mengumpulkan informasi dari berbagai halaman web. program crawling yang bisa di sebut juga "web spider" yang secara sistematis mengunjungi halaman-halaman web dan mengumpulkan data seperti teks, gambar dan metadata.

proses crawling ini biasanya digunakan oleh mesin search engine untuk mengindeks konten web, sehingga ketika orang melakukan pencarian, mesin pencari dapat dengan cepat memberikan konten yang relevan.

#proses melakukan crawling
##tool atau library


In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import time

- BeautifulSoup berfungsi untuk parsing HTML dan ekstraksi data
- request berfungsi untuk mengirim permintaan HTTP
- pandas berfungsi untuk membaca dan menganalisis data yang disimpan
- time berfungsi untuk memberikan delay agar server tidak overload

#code program



In [2]:
def replace_text(txt):
  txt = txt.replace('Baca berita tanpa iklan.', '')
  txt = txt.replace('\xa0', '')
  txt = txt.replace('\n', '')
  return txt

fungsi ini berfungsi untuk menghapus text yang tidak dibutuhkan


In [3]:
def scraping(soup):
    isi_berita = {}

    title = soup.find('h1', {'class': 'read__title'})
    isi_berita["judul"] = title.text.strip() if title else "Title not found"

    rplc = []
    berita = soup.find('div', {'class': 'read__content'})
    if berita:
        for paragraph in berita.find_all('p'):
            rplc.append(replace_text(paragraph.text))
        isi_berita["isi"] = " ".join(rplc)
    else:
        isi_berita["isi"] = "Content not found"

    date = soup.find('div', {'class': 'read__time'})
    isi_berita["tanggal"] = date.text.strip() if date else "Date not found"

    category = soup.select_one('body > div.wrap > div.container.clearfix > div:nth-child(4) > div > h3 > ul > li:nth-child(3) > a > span')
    isi_berita["kategori"] = category.text.strip() if category else "Category not found"
    return isi_berita


fungsi ini untuk proses scraping data dengan mengambil judul, pembersihan isi, tanggal berita, dan kategori berita

In [4]:
def linked_news(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return bs(response.text, "html5lib")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

fungsi linked_news ini untuk mengambil isi html dalam web yang membutuhkan library request dan BeautifulSoup

In [5]:
def link_berita(soup):
    if soup:
        konten = soup.find('div', {'class': 'articleList -list'})
        if konten:
            articles = konten.find_all("div", class_="articleItem")
            print(f"Found {len(articles)} articles on this page.")  # Debugging print
            return articles
    print("No articles found.")  # Debugging print
    return []

fungsi link_berita untuk mengambil semua berita yang berada di halaman hompage web

In [6]:
def start(page=1):
    base_url = 'https://indeks.kompas.com/?site=news&page='
    news = []

    for current_page in range(1, page + 1):
        url = base_url + str(current_page)
        print(f"Fetching page: {url}")  # Debugging print
        soup = linked_news(url)
        if soup:
            articles = link_berita(soup)
            for article in articles:
                new_link = article.find('a')['href']
                soup_berita = linked_news(new_link)
                if soup_berita:
                    hasil = scraping(soup_berita)
                    news.append(hasil)
        time.sleep(1)
    return news

fungsi start ini untuk perulangan mengambil url berita pada halaman homepage, dan juga menggunakan funtion yang telah dibuat sebelumnya.

kemudian ada program untuk memasukkan berapa halaman yang ingin di scraping

ini fungsi untuk memasukkannya ke dalam data frame

In [7]:
page = int(input('Masukkan jumlah halaman: '))
berita = start(page)

if berita:
    # Create a DataFrame and display it
    dataFrame = pd.DataFrame(berita)
    print(dataFrame)

    # Save DataFrame to a CSV file
    csv_filename = 'berita_kompas.csv'
    dataFrame.to_csv(csv_filename, index=False, encoding='utf-8-sig')
    print(f"Data has been saved to {csv_filename}")
    
    # Save DataFrame to a Pickle file
    pkl_filename = 'berita_kompas.pkl'
    dataFrame.to_pickle(pkl_filename)
    print(f"Data has also been saved to {pkl_filename}")
else:
    print("No news data was collected.")


Fetching page: https://indeks.kompas.com/?site=news&page=1
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=2
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=3
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=4
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=5
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=6
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=7
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=8
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=9
Found 15 articles on this page.
Fetching page: https://indeks.kompas.com/?site=news&page=10
Found 15 articles on this page.
                                                 judul  \
0    Bilang "Rakyat Jelata" Saa